In [ ]:
import pandas as pd

orders = pd.read_csv('olist_orders_dataset.csv')
items  = pd.read_csv('olist_order_items_dataset.csv')
customers = pd.read_csv('olist_customers_dataset.csv')

orders.head(), items.head(), customers.head()


In [ ]:
orders['order_purchase_timestamp'] = pd.to_datetime(
    orders['order_purchase_timestamp'],
    errors='coerce'
)


In [ ]:
orders[['order_purchase_timestamp']].info()


In [ ]:
orders_ok = orders[orders['order_status'] == 'delivered'].copy()



In [ ]:
orders_ok['order_month'] = orders_ok['order_purchase_timestamp'].dt.to_period('M').astype(str)

In [ ]:
orders_ok = orders_ok[['order_id', 'customer_id', 'order_purchase_timestamp', 'order_month', 'order_status']]
items = items[['order_id', 'product_id', 'seller_id', 'price', 'freight_value']]
customers = customers[['customer_id', 'customer_state']]


In [ ]:
base = orders_ok.merge(items, on='order_id', how='inner')


In [ ]:
base = base.merge(customers, on='customer_id', how='left')


In [ ]:
base[['order_id','order_month','price','customer_state']].head()
base.shape


In [ ]:
base.to_csv('olist_powerbi_final.csv', index=False)


In [ ]:
kpi_mensal = (
    base.groupby('order_month', as_index=False)
        .agg(
            faturamento=('price','sum'),
            pedidos=('order_id','nunique'),
            frete=('freight_value','sum')
        )
        .sort_values('order_month')
)

kpi_mensal['ticket_medio'] = kpi_mensal['faturamento'] / kpi_mensal['pedidos']
kpi_mensal.to_csv('kpi_mensal.csv', index=False)

kpi_mensal.head()
